In [28]:
import simpy
import random
import itertools
import numpy as np

'''
Pessoas vão utilizar um vestiário
Estão em fila, cada uma com ([n_min,n_max]) peças para experimentar
Pessoa leva um tempo para entrar no vestiário, leva um tempo (t_retirar) para retirar a roupa, (t_colocar) para colocar a roupa e (t_look) para ver se gostou da roupa
Se a pessoa gostar da roupa, ela retira a roupa nova (t_retirar), recoloca a roupa original (t_colocar) e sai do vestiário para comprar
Se a pessoa não gostar da roupa, ela retira a roupa nova (t_retirar), e experimenta outra roupa (t_colocar) que levou para experimentar.
Se nenhuma servir, ela sai do vestiário e não compra nada. 
'''

class Vestiario:
    def __init__(self, env, provadores) -> None:
        self.env = env
        self.provadores = simpy.Resource(env,capacity=provadores)

class Pessoa:
    def __init__(self, env, nome, pecas, vestiario) -> None:
        self.env = env
        self.nome = nome
        self.vestiario = vestiario
        self.pecas = random.randint(*pecas)
        self.t_chegar = max(0.10,np.random.normal(0.15,0.05))
        self.t_retirar = max(0.5,np.random.normal(1,0.5))
        self.t_colocar = max(1,np.random.normal(2,1))
        self.t_look = max(0.15,np.random.normal(0.5,0.25))

    def experimenta_roupas(self):

        yield self.env.timeout(self.t_chegar)
        print(f'{self.nome} chegou ao vestiário em {self.env.now:.2f} com {self.pecas} roupas para experimentar')

        with self.vestiario.provadores.request() as req:
            yield req

            print(f'{self.nome} acessou o vestiário em {self.env.now:.2f}')
            yield self.env.timeout(self.t_retirar)
            print(f'{self.nome} retirou a roupa em {self.env.now:.2f}')

            yield self.env.timeout(self.t_colocar)
            print(f'{self.nome} colocou a roupa em {self.env.now:.2f}')

            while self.pecas > 0:

                yield self.env.timeout(self.t_look)

                if random.choice([True, False]):
                    print(f'{self.nome} gostou da roupa em {self.env.now:.2f}')
                    yield self.env.timeout(self.t_retirar)
                    print(f'{self.nome} retirou a roupa nova em {self.env.now:.2f}')

                    yield self.env.timeout(self.t_colocar)
                    print(f'{self.nome} recolocou a roupa original em {self.env.now:.2f}')

                    print(f'{self.nome} saiu do vestiário para comprar em {self.env.now:.2f}')
                    break
                else:

                    print(f'{self.nome} não gostou da roupa em {self.env.now:.2f}')
                    
                    yield self.env.timeout(self.t_retirar)
                    print(f'{self.nome} retirou a roupa que não gostou em {self.env.now:.2f}')

                    yield self.env.timeout(self.t_colocar)
                    print(f'{self.nome} colocou outra roupa em {self.env.now:.2f}')

                    self.pecas -= 1
                    print(f'({self.nome} tem {self.pecas} restantes para experimentar)')

            if self.pecas == 0:
                print(f'{self.nome} não gostou de nenhuma peça. Saindo do vestiário em {self.env.now:.2f}')

def setup(env, provadores, pecas, pessoas):
    vestiario = Vestiario(env,provadores=provadores)

    pessoas = [Pessoa(env,nome=f'Pessoa {i}',pecas=pecas, vestiario=vestiario) for i in range(pessoas)]
    for pessoa in pessoas:
        env.process(pessoa.experimenta_roupas())

    env.run()

PESSOAS = 5
PROVADORES = 3
PECAS = [1,5]

env = simpy.Environment()
setup(env,provadores=PROVADORES,pecas=PECAS,pessoas=PESSOAS)

    

Pessoa 0 chegou ao vestiário em 0.10 com 2 roupas para experimentar
Pessoa 0 acessou o vestiário em 0.10
Pessoa 3 chegou ao vestiário em 0.12 com 3 roupas para experimentar
Pessoa 3 acessou o vestiário em 0.12
Pessoa 2 chegou ao vestiário em 0.16 com 4 roupas para experimentar
Pessoa 2 acessou o vestiário em 0.16
Pessoa 1 chegou ao vestiário em 0.17 com 5 roupas para experimentar
Pessoa 4 chegou ao vestiário em 0.19 com 5 roupas para experimentar
Pessoa 2 retirou a roupa em 0.82
Pessoa 3 retirou a roupa em 1.22
Pessoa 0 retirou a roupa em 1.49
Pessoa 2 colocou a roupa em 1.82
Pessoa 2 gostou da roupa em 2.47
Pessoa 0 colocou a roupa em 3.00
Pessoa 2 retirou a roupa nova em 3.13
Pessoa 3 colocou a roupa em 3.45
Pessoa 0 gostou da roupa em 3.64
Pessoa 3 não gostou da roupa em 3.96
Pessoa 2 recolocou a roupa original em 4.13
Pessoa 2 saiu do vestiário para comprar em 4.13
Pessoa 1 acessou o vestiário em 4.13
Pessoa 0 retirou a roupa nova em 5.03
Pessoa 3 retirou a roupa que não gostou em 